# Statistics 2 : OpenSubtitle Automatic Subset

**Input :** Automatic_full.csv

**Main :** Extract dialogues id. Do the same step as in Manual_information notebook

**Output :** Atomatic_subset.pkl & Automatic_subset_info.pkl & Automatic_subset_info_vader.pkl


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time
import nltk
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns

In [3]:
Emotional = ['afraid',#Fear
            'angry',#anger
            'annoyed',#anger
            'anticipating',#Anticipation
            'anxious',#Fear
            'apprehensive',#Fear
            'ashamed',#Disapproval
            'caring',#love
            'confident',#Trust
            'content',#Joy
            'devastated',#Sadness
            'disappointed',#Sadness
            'disgusted',#Disgust
            'embarrassed',#Disapproval
            'excited',#Joy
            'faithful',#Submission
            'furious',#anger
            'grateful',#Trust
            'guilty',#Remorse
            'hopeful',#Anticipation
            'impressed',#Suprise
            'jealous',#Agressiveness
            'joyful',#Joy
            'lonely',#Sadness
            'nostalgic',#Sadness
            'prepared',#Anticipation
            'proud',#Optimism
            'sad',#Sadness
            'sentimental',#Sadness
            'surprised',#Suprise
            'terrified',#Fear
            'trusting']#Trust
neutral = [ 'agreeing', # neutral
            'acknowledging', # neutral 
            'encouraging', # neutral 
            'consoling', # neutral 
            'sympathizing', # neutral 
            'suggesting', # neutral 
            'questioning', # neutral 
            'wishing', # neutral
            'neutral']
neutral_num   = ["32","33","34","35","36","37","38","39","40"]
Emotional_num = ["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

# Array type of the labels
labels = np.array(['afraid',#Fear
            'angry',#anger
            'annoyed',#anger
            'anticipating',#Anticipation
            'anxious',#Fear
            'apprehensive',#Fear
            'ashamed',#Disapproval
            'caring',#love
            'confident',#Trust
            'content',#Joy
            'devastated',#Sadness
            'disappointed',#Sadness
            'disgusted',#Disgust
            'embarrassed',#Disapproval
            'excited',#Joy
            'faithful',#Submission
            'furious',#anger
            'grateful',#Trust
            'guilty',#Remorse
            'hopeful',#Anticipation
            'impressed',#Suprise
            'jealous',#Agressiveness
            'joyful',#Joy
            'lonely',#Sadness
            'nostalgic',#Sadness
            'prepared',#Anticipation
            'proud',#Optimism
            'sad',#Sadness
            'sentimental',#Sadness
            'surprised',#Suprise
            'terrified',#Fear
            'trusting',#Trust
            'agreeing', # neutral
            'acknowledging', # neutral 
            'encouraging', # neutral 
            'consoling', # neutral 
            'sympathizing', # neutral 
            'suggesting', # neutral 
            'questioning', # neutral 
            'wishing', # neutral
            'neutral']) # neutral 
# MAPPING in array type 
plutchik = np.array(['Fear',
            'anger',
            'anger',
            'Anticipation',
            'Fear',
            'Fear',
            'Disapproval',
            'love',
            'Trust',
            'Joy',
            'Sadness',
            'Sadness',
            'Disgust',
            'Disapproval',
            'Joy',
            'Submission',
            'anger',
            'Trust',
            'Remorse',
            'Anticipation',
            'Suprise',
            'Agressiveness',
            'Joy',
            'Sadness',
            'Sadness',
            'Anticipation',
            'Optimism',
            'Sadness',
            'Sadness',
            'Suprise',
            'Fear',
            'trust',
            'neutral',
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral',
            'neutral']) # neutral 
Column_Emo = ["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40"]

In [ ]:
# Read Automatic data set
data_TA= pd.read_csv(r"Automatic_full.csv", usecols = [1,2,5,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]) 

# Read Manual subset
Solution = pd.read_pickle("Manual_subset.pkl")

In [11]:
# Match Id_index
l1  = Solution.Num_dialog.unique().tolist()
data_TA = data_TA[data_TA["dialogue_id"].isin (l1)]

In [ ]:
# Force type
data_TA["text"] = data_TA.text.astype(str)
# Get the token of each sentences in extra column
data_TA["Token"] = data_TA["text"].apply(nltk.word_tokenize)
# Get the lenght of each sentences in extra column
data_TA["Size"] = data_TA["Token"].apply(len)
# Drop Useless Column
data_TA.drop(columns=['Token'], inplace=True)
data_TA.drop(columns=['text'], inplace=True)
data_TA.drop(columns=['turn'], inplace=True)
# Creat an extra row that will be used for the number of turn
data_TA["Nbr_Row"] = 1
# Groupby id
result = data_TA.groupby(["dialogue_id"]).mean().copy()
# Get number of turn per dialog
result["Nbr_Row"] = data_TA.groupby(["dialogue_id"]).Nbr_Row.sum().copy()
# Get prominent Category
result["Prominent_score"] = result[Column_Emo].max(axis=1)
result["Prominent_EmoBert"] =result[Column_Emo].idxmax(axis=1)
result["Prominent_EmoBert"] = result.Prominent_EmoBert.astype(int)
result["Prominent_EmoBert"] = result["Prominent_EmoBert"].apply(lambda x : labels[x])


# Get prominent plutchik 
result["Prominent_Plutchik"] = result.Prominent_EmoBert.apply(lambda x : plutchik[labels.tolist().index(x)])
#Neutrality score
result["Neutrality_score"] = result[neutral_num ].sum(axis=1)
#Emotionality score
result["Emotionality_score"] = result[Emotional_num].sum(axis=1)
# Drop Useless column
result.drop(columns=Column_Emo, inplace =True)

In [ ]:
#Pickle
result.to_pickle("Automatic_subset_info.pkl")

# Build Vader

In [ ]:
data_TA = pd.read_pickle("Automatic_subset.pkl")

In [13]:
data_TA = data_TA[["dialogue_id","text"]]

In [16]:
data_TA["text"] = data_TA.text.astype(str)
data_TA

C:\Users\stan\AppData\Local\Continuum\anaconda3\envs\ada\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,dialogue_id,text
2438,489,"_ Kove : Trent , report ."
2439,489,Have you taken out the target ?
2440,489,Trent : Negative . He 's got your burn book...
2441,489,If that gets into the wrong hands- Kove : Y...
2442,489,"Alphonse , do you have a shot ?"
...,...,...
52808179,8819128,Good point . I 'll just go check on him .
52808180,8819129,ROZ : It 's weird .
52808181,8819129,"My skin tastes kinda salty . BULLDOG : Oh ,..."
52808182,8819129,"That wasn 't me , Bulldog ."


In [ ]:
data_TA["text"] = data_TA.text.apply(lambda x : x.strip())
# Vader Analyse
analyzer = SentimentIntensityAnalyzer()
data_TA["Vader"] = data_TA.text.apply(lambda x: analyzer.polarity_scores(x)['compound'])
# Drop useless column
data_TA.drop(columns=['text'], inplace=True)
# Get the mean vader score per dialog
Vader = data_TA.groupby(["dialogue_id"]).mean().copy()

In [ ]:
result2 =  pd.read_pickle("Automatic_subset_info.pkl")
result2["Vader"]= Vader
# Information with vader
result2.to_pickle("Automatic_subset_info_vader.pkl")